In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import sparse
from sklearn.model_selection import train_test_split
from datetime import datetime
import statistics
import math

In [2]:
# importing required dataset - u.data
cols = ['userId', 'movie id', 'rating', 'timestamp']
df = pd.read_csv(r'F:\Harshi College\MS DA\Machine Learning\lab\recommender system - movie lens\ml-100k\u.data',sep='\t', names=cols)
print(df.head)
print(max(df['userId']), min(df['userId']), max(df['movie id']), min(df['movie id']))

<bound method NDFrame.head of        userId  movie id  rating  timestamp
0         196       242       3  881250949
1         186       302       3  891717742
2          22       377       1  878887116
3         244        51       2  880606923
4         166       346       1  886397596
...       ...       ...     ...        ...
99995     880       476       3  880175444
99996     716       204       5  879795543
99997     276      1090       1  874795795
99998      13       225       2  882399156
99999      12       203       3  879959583

[100000 rows x 4 columns]>
943 1 1682 1


In [3]:
df.shape

(100000, 4)

##### table creation - training data

In [4]:
# Function to insert row in the dataframe
def Insert_row_(row_number, df, row_value):
    # Slice the upper half of the dataframe
    df1 = df[0:row_number]
  
    # Store the result of lower half of the dataframe
    df2 = df[row_number:]
  
    # Insert the row in the upper half dataframe
    df1.loc[row_number]=row_value
  
    # Concat the two dataframes
    df_result = pd.concat([df1, df2])
  
    # Reassign the index labels
    df_result.index = [*range(df_result.shape[0])]
  
    # Return the updated dataframe
    return df_result

In [5]:
def tableCreation():
    # converting training data into df
    trTable = training_data.pivot(index='userId', columns='movie id', values='rating')
    # Replace NaN Values with Zeros in Pandas DataFrame
    trTable = trTable.replace(np.nan, 0)
    
    # col validation until last but one column
    for i in range(1, max(df['movie id'])+1):
        try:
            if(i-1 == trTable.columns.get_loc(i)):
#                 print(i)
                continue

        except KeyError:
#             print(i, "not foundddd=========")
            trTable.insert(i-1, i,  [0] * len(trTable[i-1]))
    
    # rows validation until last but one column

    # Get row-index values of Pandas DataFrame as list
    rowIndList = trTable.index.values.tolist()
    for i in range(1, max(df['userId'])):
        if(rowIndList[i-1] == i):
            continue

        else:
#             print(i, "not foundddd\\\\\\\\\\\\\\\\\\\\\\")
            row_number = i
            row_value = [0] * len(trTable.iloc[i-1])
            trTable_df = Insert_row_(row_number, trTable, row_value)
            trTable = trTable_df

    trTable.to_csv("trTable_dfuooo.csv")
    
    return trTable


##### find cosine similarity between the users in training data

In [6]:
def findSimilarity(table_tr_data):
    trTable = table_tr_data 
#     list that will contain the sim values whoch will be converted to df
    simTrList = []
    
    # finding the len of df to iterate
    lenOfTrTable = len(trTable[1])+1

    # Pandas Dataframe.to_numpy() – Convert dataframe to Numpy array
    List1 = trTable.to_numpy()

    for everyRow in range(1, lenOfTrTable):
    # iterating row by row in df
        List2 = (trTable.loc[everyRow]).to_numpy()
        similarity_scores = List1.dot(List2)/ (np.linalg.norm(List1, axis=1) * np.linalg.norm(List2))
       
        simTrList.append(similarity_scores)
        
    simTrDf = pd.DataFrame(simTrList)
#     writing df into csv
    simTrDf.to_csv("similarity btwn users - training data.csv")
    return simTrDf



##### table creation - testing data

In [7]:
def testTableCreation():
    # table craetion 
    (tstTable) = testing_data.pivot(index='userId', columns='movie id', values='rating')
    tstTable = tstTable.replace(np.nan, 0)
    tstTable.to_csv("preetstTable_dfuooo.csv")
    # col validation until last but one column
    for i in range(1, max(df['movie id'])+1):
        try:
            if(i-1 == tstTable.columns.get_loc(i)):
    #                 print(i)
                continue

        except KeyError:
#             print(i, "not foundddd=========")
            tstTable.insert(i-1, i,  [0] * len(tstTable[i-1]))

    # rows validation until last but one column

    # Get row-index values of Pandas DataFrame as list
    rowIndList = tstTable.index.values.tolist()
    
    flag = 0
    for i in range(1, len(tstTable)+1):
        if i not in rowIndList:
            flag = 1
#             print(i)
            row_number = i
            row_value = [0] * len(tstTable.iloc[i-1])
            tstTable_df = Insert_row_(row_number, tstTable, row_value)
    

    #copy the new df to old df only if missing rows are added
    if(flag == 1):
        tstTable = tstTable_df
    
    tstTable.to_csv("tstTable_dfuooo.csv")
    return tstTable


In [8]:
def findKNN(everyUser, everyMovie):
    sortedUserCol = sorted(list(sim_btwn_users[everyUser-1]), reverse=True)
    simValList, simValIndList = [], []
    for j in range(30): 
#         extracting top 30 NN
        simVal = sortedUserCol[j]
        simValList.append(simVal)
        
#         extracting the index of top 30 NN 
        simValInd = list(sim_btwn_users[everyUser-1]).index(sortedUserCol[j])
        simValIndList.append(simValInd)
        
    ratingsValList = []
    for NN_user in range(0,len(simValIndList)):
        try:
#         finding ratings for NNusers and movieId
            ratingsVal = tstTable[simValIndList[NN_user]][everyMovie]

        except:
            ratingsVal = 0

        finally:
#                 print(f"{ratingsVal} for {simValIndList[NN_user]}")
            ratingsValList.append(ratingsVal)
            
                   
#     print(f"\n30NN for user:{everyUser}\n\nsimValList\n{simValList}\n\nsimValIndList\n{simValIndList}\n\nratingsValList\n{ratingsValList}\n\n")
        
    return simValList, simValIndList, ratingsValList
    
# # findKNN(everyUser, everyMovie)
# for i in range(len(findKNN(1, 1))):
#     print(findKNN(1, 1)[i])


In [9]:
# workinf and modifes code
def predictRatings(everyUser, everyMovie, everyRating):
    
    everyMovie = everyMovie + 1
#     print("\n\ncomputing below for",everyUser, everyMovie, everyRating)
#     everyUser = everyUser - 1
    sim, ind, rat = findKNN(everyUser, everyMovie)
#     print(rat)

    diffList = []
    
    if(len(set(rat)) == 1):
        actual = everyRating
        pred = 0
        diff = (actual - pred)**2
        diffList.append(diff)
        
    else:
        actual = everyRating
        simDr = 0
        simNr = 0
        for everyVal in range(len(rat)):
            if(rat[everyVal] != 0):
                simNr = (rat[everyVal] * sim[everyVal]) + simNr
                simDr = simDr + sim[everyVal]
                pred = simNr/simDr
                
        diff = (actual - pred)**2
        diffList.append(diff)
    
    rmse = math.sqrt(statistics.mean(l))
    return rmse
        




##### function calling from __main__

In [10]:
# workingg codeee
from sklearn.model_selection import train_test_split
rmseChunkList = []
for rs in range(10,15):
    countRatTest = 0
    outer, inner = [], []
    training_data, testing_data = train_test_split(df, test_size=0.2, random_state=rs)

    print(f"No. of training examples: {training_data.shape[0]}")
    print(f"No. of testing examples: {testing_data.shape[0]}")
    
    now = datetime.now().time() # time object
    print("start time =", now)

    table_tr_data = tableCreation()
    sim_btwn_users = findSimilarity(table_tr_data)

    diffList = []
    tstTable = testTableCreation()
    # finding not null values in df
    for everyUser in range(1, len(tstTable)+1):
        try:
            idx = pd.Index(tstTable.loc[everyUser])
            
        except:
    #         some userId might not exist. so handling such scenarios
            continue

    #     diffList = []=================
        
        for everyMovie in range(0, len(list(idx))):
            everyRating = list(idx)[everyMovie]
            outer.append([everyUser, everyMovie, everyRating,"\n"])
            if(everyRating != 0):
                inner.append([everyUser, everyMovie, everyRating,"\n"])
                countRatTest += 1
                #predict for val
    #             call findKNN function
    #             findKNN(everyUser, everyMovie)
                diff = predictRatings(everyUser, everyMovie, everyRating)
                diffList.append(diff)

    print(f"\nnumber of values for which predictions are made : {len(diffList)}\n") 
    print("\ncountRatTest\n",countRatTest)
    import math
    import statistics
    rmse = statistics.mean(diffList)  
    rmse = math.sqrt(rmse)
    print("\nthe rmse is",rmse)

    now = datetime.now().time() # time object
    print("\nend time =", now)
    print("\n===========================================================\n")
    rmseChunkList.append(rmse)

print(f"the rmse of the model is {statistics.mean(rmseChunkList)}")

No. of training examples: 80000
No. of testing examples: 20000
start time = 08:53:19.134652


C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3457: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  exec(code_obj, self.user_global_ns, self.user_ns)


NameError: name 'l' is not defined

In [ ]:
the rmse of the model is 1.9009263695916123

In [ ]:
https://www.statology.org/rmse-python/#:~:text=To%20calculate%20the%20RMSE%20between,What%20is%20this%3F&text=The%20RMSE%20turns%20out%20to%20be%202.4324.